Welcome to module 2.2! In the last module, we investigated how to use `scikit-learn` to build a simple Naive Bayes model to perform sentiment analysis. In this notebook we will look behind the scenes: we will build our own Naive Bayes model from scratch. We will reuse some of the code we wrote in the previous notebook - but this time, we'll write the maths from scratch.

## ❓ Pre-module quiz

Why is Naive Bayes "naive"?

A. Because it's the most basic, i.e. "naive" classifier we can build

B. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are independent of each other

C. Because the guy who invented it tought it was a cool name

D. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are dependent of each other

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>The correct answer is B - Naive Bayes assumes that the probability of finding a certain word is independent from the probability of finding another word. So, for example, in the domain of movie reviews, it assumes that the probabilities of finding the words <code>Indiana</code> and <code>Jones</code> are not correlated, even if in practice we know that this is not the case.</p>

</details> 


# Python for Computational Linguists 1.2: Breaking down Naive Bayes

## Introduction

### Recap of last notebook

Module 2.1 introduced lots of new concepts and libraries - pre-processing, feature vectors, `numpy`, and so on. In fact, we have seen how to import data from a research dataset, how to clean them by removing punctuation and stop-words, how to use `numpy` to prepare test and training data for a model, and how to use `scikit-learn` to train, test and evaluate a simple Naive Bayes classifier for sentiment analysis.

However, we didn't look 'behind the scenes' at how Naive Bayes actually works. This module will guide you through your first hand-written machine learning model, by showing you how to write the maths for Naive Bayes yourself. We will introduce the concept of "class", and we'll debug and improve our model by adding smoothing.

If you're interested, you can also just go and look the code of `scikit-learn`'s implementation [here](https://github.com/scikit-learn/scikit-learn/blob/0fb307bf3/sklearn/naive_bayes.py#L669)!

### Naive Bayes refresher

> Note: this section borrows heavily from the Naive Bayes chapter of the lecture notes. Please refer to them or to the J&M for more details.

Naive Bayes is a simple classifier based on two assumptions:
- The **bag-of-words** assumption: word ordering doesn't matter. We represent each document in our dataset as a list of pairs $(word_i,frequency_i)$.
- The **conditional independence** assumption: the probability of one word appearing in a sentence is by no means correlated to the occurrence of another word.

These two assumptions heavily simplify our model, since they completely disregard grammar and and eventual domain knowledge (e.g. the `Indiana Jones` example in the pre-module quiz); but as we know now, they allow us to build a surprisingly efficient classifier.

Let's quickly go through the maths of Naive Bayes. Remember that given a document $d$ and a set of classes $C$, the probability the document belong to a class $c \in C$ is $\hat{c}$, and is defined as:

$$ \hat{c} = \text{argmax}_{c \in C} P(c \mid d)$$

However, the Bayes Rule tells us that

$$ P(c \mid d) = \frac{P(c) \ P(d \mid c)}{P(d)} $$

Allowing us to rewrite 

$$\begin{align} 
\hat{c} &= \text{argmax}_{c \in C} P(c \mid d) \\
        &= \text{argmax}_{c \in C} \frac{P(c) \ P(d \mid c)}{P(d)}
\end{align}$$

However, the probability $d$ is constant for each class $c$, hence we can remove it, leaving only:

$$
\hat{c} = \text{argmax}_{c \in C} 
    \underbrace{P(c) \ P(d \mid c)}_\text{likelihood}
    \underbrace{P(d)}_\text{prior}
$$

Where the $prior$ is the **prior probability** of the class $c$ and the $likelihood$ is the probability of finding $d$ given the class $c$.

Using words as features, we can represent $d$ as a list of words $w_1, \dots , w_n$, hence 

$$
\hat{c} = \text{argmax}_{c \in C} 
    \underbrace{P(c) \ P(w_1, \dots , w_n \mid c)}_\text{likelihood} 
    \underbrace{P(d)}_\text{prior}
$$

However, $P(w_1, \dots , w_n \mid c)$ may be prohibitively hard to calculate, since we would need to estimate the probability of every possible combination of words. Here, the **bag-of-words** assumption comes to the rescues, assuming the probability of the words (i.e. features) are independent, allowing us to finally rewrite

$$
\begin{align}
\hat{c} &= \text{argmax}_{c \in C} 
       \underbrace{P(d)}_\text{prior}
       \underbrace{P(c) \ P(w_1 \mid c) \times \dots \times P(w_n \mid c)}_\text{likelihood} 
       \\
       &= \text{argmax}_{c \in C} 
       \underbrace{P(d)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} 
\end{align}
$$

What does all of this mean in practice? Well, that if we have a document $d$, all we need to know to classify it is:
- The *priors*, i.e. the probability of document $d$ to belong to each class $c$
- The *likelihoods*, i.e. the probabilities for each word $w_i$ of the document to belong to each class $c$.

## Naive Bayes: a simple implementation

Let's begin with a simple example from the Post Lecture exercises (taken from J&M-3, exercise 4.2). Given the following short movie reviews, each labeled with a genre, either comedy or action:

| review                      | class  |
|-----------------------------|--------|
| fun, couple, love, love     | comedy |
| fast, furious, shoot        | action |
| couple, fly, fast, fun, fun | comedy |
| curious, shoot, shoot, fun  | action |
| fly, fast, shoot, love      | action |

And a new document D: 

| review                     | class  |
|----------------------------|--------|
| fast, couple, shoot, fly   | ?      |

Compute the most likely class for D.

Let's start by saving our documents in some vectors:

In [ ]:
train_docs = [
    ['fun', 'couple', 'love', 'love'],
    ['fast', 'furious', 'shoot'],
    ['couple', 'fly', 'fast', 'fun', 'fun'],
    ['curious', 'shoot', 'shoot', 'fun'],    
    ['fly', 'fast', 'shoot', 'love']]

train_labels = ['comedy', 'action', 'comedy', 'action', 'action']

Remember what we needed to do? The first step is to compute the **priors**. Let's do that with a simple function:

In [ ]:
# What are our classes?
classes = set(train_labels)
print(classes)

# initialise the priors
priors = {}
for _class in classes:
    priors[_class] = 0

# count how many train example in each class
for _class in classes:
    for label in train_labels:
        if _class == label:
            priors[_class] += 1

print(priors)

> **<h3>💻 Try it yourself!</h3>**

Now the priors are not *normalised*, i.e. we have to bring each prior in the range $[0,1]$. Can you do that in the following cell?

In [ ]:
# write your code here

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
for _class in classes:
    priors[_class] = priors[_class] / len(train_labels)
    </code></pre></p>

</details> 

If you've got the priors correct, you should have $P(comedy) = 0.4$ and $P(action)=0.6$.

In [ ]:
print(priors)

> **<h3>💻 Try it yourself!</h3>**

Now let's wrap everything nicely into a function. Can you complete the cell below?

In [ ]:
def compute_priors(labels):
    '''
    Computes the priors for a set of labels.
    '''

    # What are our classes?
    classes = set(labels)

    # initialise the priors
    priors = {}

    # ...?

    # count how many train example in each class
    
    # ...?
    
    # normalise the priors
    
    # ...?
    
    return priors

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def compute_priors(labels):
    '''
    Computes the priors for a set of labels.
    '''
    
    # What are our classes?
    classes = set(labels)

    # initialise the priors
    priors = {}
    for _class in classes:
        priors[_class] = 0

    # count how many train example in each class
    for _class in classes:
        for label in labels:
            if _class == label:
                priors[_class] += 1
    
    # normalise the priors
    for _class in classes:
        priors[_class] = priors[_class] / len(labels)
        
    return priors
    </code></pre></p>

</details> 

In [ ]:
priors = compute_priors(train_labels)
print(priors)

### Computing the probabilities

### The training function

### Object-oriented programming in a nutshell

#### Your first class: `Dataset`

#### The `NaiveBayes` class

### Adding 1-smoothing

## Re-classify the *Thumbs up?* dataset

### Recap of loader functions

### Applying our model

### Improving the results

### Differences between our model and `scikit-learn`

## Wrapping up